In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle

from textblob import TextBlob
from googletrans import Translator

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, Dropout, Bidirectional, LSTM, GRU, Input, GlobalMaxPooling1D, LayerNormalization
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import Sequential 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pyvi import ViTokenizer, ViUtils
from vncorenlp import VnCoreNLP


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 1. Đọc dữ liệu từ CSV
df = pd.read_csv('sentiments_v2_6001_7000.csv')

# In ra tên các cột để chắc chắn rằng cột 'comment' và 'sentiment' tồn tại
print(df.columns)

# Nếu không có cột 'sentiment', tạo nhãn cảm xúc
if 'sentiment' not in df.columns:
    def label_sentiment(comment):
        if 'good' in comment or 'love' in comment:
            return 1  # Tích cực
        else:
            return 0  # Tiêu cực
    df['sentiment'] = df['comment'].apply(label_sentiment)

# 2. Tiền xử lý dữ liệu
max_features = 10000  # Giới hạn số từ để giữ lại
max_len = 200  # Độ dài tối đa của mỗi comment

# Tokenizer để chuyển văn bản thành các chỉ số
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['comment'])  # Tạo từ điển các từ trong dữ liệu (cột 'comment')
x_data = tokenizer.texts_to_sequences(df['comment'])  # Chuyển đổi văn bản thành chỉ số
x_data = pad_sequences(x_data, maxlen=max_len)  # Padding cho các câu có độ dài khác nhau

# 3. Mã hóa nhãn cảm xúc
y_data = df['sentiment'].values  # Nhãn cảm xúc (1 cho tích cực, 0 cho tiêu cực)

# 4. Xây dựng mô hình phân loại cảm xúc
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=max_len),
    LSTM(64, dropout=0.2, recurrent_dropout=0.2),
    Dense(1, activation='sigmoid')
])

# 5. Compile mô hình
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 6. Huấn luyện mô hình
model.fit(x_data, y_data, epochs=5, batch_size=64, validation_split=0.2)

# 7. Dự đoán cảm xúc cho tất cả các comment trong file CSV
predictions = model.predict(x_data)

# Chuyển đổi dự đoán thành nhãn (1 là tích cực, 0 là tiêu cực)
predicted_labels = [1 if pred >= 0.5 else 0 for pred in predictions]

# Thêm kết quả dự đoán vào DataFrame
df['predicted_sentiment'] = predicted_labels

# 8. Lưu kết quả vào file CSV mới
df.to_csv('reviews_with_sentiment_predictions.csv', index=False)

# 9. Hiển thị kết quả
print(df[['comment', 'sentiment', 'predicted_sentiment']])


Index(['userid', 'mtime', 'rating_star', 'comment'], dtype='object')


c:\Users\admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 94ms/step - accuracy: 0.8959 - loss: 0.5870 - val_accuracy: 1.0000 - val_loss: 0.0456
Epoch 2/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - accuracy: 0.9918 - loss: 0.0588 - val_accuracy: 1.0000 - val_loss: 0.0035
Epoch 3/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - accuracy: 0.9959 - loss: 0.0267 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 4/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9968 - loss: 0.0212 - val_accuracy: 1.0000 - val_loss: 0.0030
Epoch 5/5
13/13 ━━━━━━━━━━━━━━━━━━━━ 1s 64ms/step - accuracy: 0.9957 - loss: 0.0278 - val_accuracy: 1.0000 - val_loss: 0.0046
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
                                               comment  sentiment  \
0    Dhshigdgiggyhudauoegvngsyhgxhgahbchkđghdgvxgbv...          0   
1              drkdrkcnkddnshdkgosqnandcxnsgogkssndkco          0   
2    Son môi rẻ, chất lượng oki nhưng không thích m...          0   
3         Giao đầy đủ và chất lượng đáng mua nha mnnmm

In [31]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Load tokenizer và model PhoBERT cho tiếng Việt
tokenizer = BertTokenizer.from_pretrained('vinai/phobert-base')
model = TFBertForSequenceClassification.from_pretrained('vinai/phobert-base')

# Tiền xử lý văn bản tiếng Việt
text = "sản phẩm này tốt "
inputs = tokenizer(text, return_tensors='tf', truncation=True, padding=True, max_length=128)

# Dự đoán cảm xúc
outputs = model(inputs)
logits = outputs.logits
predicted_class = tf.argmax(logits, axis=-1).numpy()

print("Predicted class:", predicted_class)

if predicted_class == 1:
    print("tích cực")
else:
    print ("tiêu cực")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'PhobertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFBertForSequenceClassification: ['roberta', 'lm_head']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializ

Predicted class: [0]
tiêu cực


In [ ]:
# Đọc file CSV
file_path = "sentiments_v2_7001_7230.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print(data.columns)

Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')


In [11]:
# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Chỉ kiểm tra nếu cột 'comment' tồn tại
if 'comment' in data.columns:
    # Khởi tạo công cụ dịch
    translator = Translator()

    # Hàm dịch và phân tích cảm xúc
    def analyze_sentiment(text):
        # Dịch văn bản sang tiếng Anh
        translated_text = translator.translate(text, src='vi', dest='en').text
        # Phân tích cảm xúc bằng TextBlob
        blob = TextBlob(translated_text)
        return blob.sentiment.polarity  # Trả về điểm cảm xúc từ -1 (tiêu cực) đến 1 (tích cực)

    # Áp dụng hàm cho 5 bình luận đầu tiên
    data['sentiment'] = data['comment'].head(5).apply(analyze_sentiment)

    # Hiển thị 5 bình luận đầu tiên với điểm cảm xúc
    print(data[['comment', 'sentiment']].head(5))
else:
    print("Cột 'comment' không tồn tại trong DataFrame.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')
                                             comment  sentiment
0             Giá rẻ. Mua xịt khử mùi ăn uóng khá ổn   0.700000
1  thơm lắm mn oi có chai đào là khi xịt hòi là h...   0.086458
2  eidighgghuđhuêd Dbddnd ĐĐ D D c chr ý nghĩa là...   0.000000
3    Thơm nha các bạn nhưng k giữ đc mùi lâu đâu nhé  -0.050000
4                         Sản phẩm giống hình xài ok   0.000000


In [12]:
# Đọc file CSV và bỏ qua các dòng lỗi
file_path = "sentiments_v2_7001_7230.csv"
data = pd.read_csv(file_path, on_bad_lines='skip')

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Giả sử cột chứa văn bản là 'text' (thay bằng tên cột tương ứng nếu khác)
if 'comment' in data.columns:
    # Tách từ cho từng dòng trong cột 'text' và lưu vào một cột mới 'tokenized_text'
    data['tokenized_text'] = data['comment'].apply(ViTokenizer.tokenize)

    output_file_path = "sentiments_tk_7001_7230.csv"
    data.to_csv(output_file_path, index=False)  # Không lưu chỉ mục (index)
    
    print(data[['comment', 'tokenized_text']].head())
else:
    print("Không tìm thấy cột 'comment' trong DataFrame. Vui lòng kiểm tra lại tên cột.")

Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')
                                             comment  \
0             Giá rẻ. Mua xịt khử mùi ăn uóng khá ổn   
1  thơm lắm mn oi có chai đào là khi xịt hòi là h...   
2  eidighgghuđhuêd Dbddnd ĐĐ D D c chr ý nghĩa là...   
3    Thơm nha các bạn nhưng k giữ đc mùi lâu đâu nhé   
4                         Sản phẩm giống hình xài ok   

                                      tokenized_text  
0            Giá rẻ . Mua xịt khử mùi ăn uóng khá ổn  
1  thơm lắm mn oi có chai đào là khi xịt hòi là h...  
2  eidighgghuđhuêd Dbddnd ĐĐ D D c chr ý_nghĩa_là...  
3    Thơm nha các bạn nhưng k giữ đc mùi lâu đâu nhé  
4                         Sản_phẩm giống hình xài ok  


In [14]:
# Danh sách các stop words tiếng Việt (có thể mở rộng thêm)
stop_words = set([
    "và", "của", "là", "đã", "đang", "nên", "nhưng", "với", "bằng", "theo", 
    "làm", "từ", "trong", "lúc", "mà", "này", "có", "chưa", "vì", "sẽ", "khi",
    "được", "không", "tôi", "bạn", "chúng", "mình", "một", "nhiều", "vài", 
    "cũng", "nào", "nếu", "hay", "kể", "vừa", "nên", "có thể", "làm", "mà"
])

# In ra tên các cột để kiểm tra
print("Các cột trong DataFrame:", data.columns)

# Giả sử cột chứa văn bản là 'comment' (thay bằng tên cột khác nếu cần)
if 'comment' in data.columns:
    # Tách từ trong cột 'comment'
    data['tokenized_comment'] = data['comment'].apply(ViTokenizer.tokenize)
    
    # Bước loại bỏ stop words
    def remove_stopwords(text):
        words = text.split()
        filtered_words = [word for word in words if word not in stop_words]
        return " ".join(filtered_words)
    
    # Áp dụng loại bỏ stop words
    data['cleaned_comment'] = data['tokenized_comment'].apply(remove_stopwords)

    # Lưu kết quả vào file CSV mới
    output_file_path = "sentiments_v2_7001_7230.csv"
    data.to_csv(output_file_path, index=False)  # Không lưu chỉ mục (index)

    print(f"Dữ liệu đã được lưu vào file: {output_file_path}")
else:
    print("Không tìm thấy cột 'comment' trong DataFrame. Vui lòng kiểm tra lại tên cột.")


Các cột trong DataFrame: Index(['userid', 'mtime', 'rating_star', 'comment', 'tokenized_comment',
       'cleaned_comment'],
      dtype='object')
Dữ liệu đã được lưu vào file: sentiments_v2_7001_7230.csv


In [8]:
# Đặt đường dẫn tới mô hình VnCoreNLP
vncorenlp_path = r'D:\Y4 HK1\VnCoreNLP\VnCoreNLP-1.1.1.jar'  # Thay bằng đường dẫn chính xác của bạn

# Khởi tạo VnCoreNLP
vncorenlp = VnCoreNLP(vncorenlp_path)

if 'comment' in data.columns:
    # Áp dụng lemmatization cho mỗi dòng trong cột 'comment'
    def lemmatize_text(text):
    # Dùng VnCoreNLP để annotate văn bản
        annotated = vncorenlp.annotate(text)
        
        # Trích xuất các từ đã được lemmatize từ dữ liệu trả về
        words = []
        for sentence in annotated.get('sentences', []):
            for token in sentence:
                # Kiểm tra nếu 'lemma' tồn tại trong token
                if 'lemma' in token:
                    words.append(token['lemma'])
                else:
                    words.append(token['form'])  # Nếu không có 'lemma', lấy từ gốc
        
        # Trả về các từ đã lemmatized
        return ' '.join(words)

# Áp dụng lemmatization cho mỗi dòng trong cột 'comment'
    data['lemmatized_comment'] = data['comment'].apply(lemmatize_text)

    # Lưu lại kết quả vào file CSV mới
    data.to_csv('lemmatized_sentiments.csv', index=False)  # Lưu vào file mới

    print("Lematization đã hoàn thành và lưu vào file lemmatized_sentiments.csv")
else:
    print("Không tìm thấy cột 'comment' trong DataFrame.")

Lematization đã hoàn thành và lưu vào file lemmatized_sentiments.csv
